## Neural Network Model Optimization
### The cells in this ipynb show just a couple of the methods I tried to optimize my model to further improve its performance from 72.8%.

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint

# Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_columns = ['EIN', 'NAME']
application_df = application_df.drop(columns=drop_columns)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_constant_high = 700
cutoff_constant_low = 20

low_bin = []
mid_bin = []
for item in application_df['APPLICATION_TYPE'].value_counts().items():
    if item[1] < cutoff_constant_high and item[1] > cutoff_constant_low:
        mid_bin.append(item[0])
    elif item[1] < cutoff_constant_low:
        low_bin.append(item[0])

# Replace in dataframe
for app in low_bin:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"LowOther")
for app in mid_bin:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"MidOther")
    
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3          27037
T4           1542
T6           1216
T5           1173
T19          1065
MidOther      777
T8            737
T7            725
LowOther       27
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_constant_high = 500
cutoff_constant_low = 2

low_bin = []
mid_bin = []

for item in application_df['CLASSIFICATION'].value_counts().items():
    if item[1] < cutoff_constant_high and item[1] >= cutoff_constant_low:
        mid_bin.append(item[0])
    elif item[1] < cutoff_constant_low:
        low_bin.append(item[0])

# Replace in dataframe
for cls in low_bin:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"LowOther")
for cls in mid_bin:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"MidOther")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000       17326
C2000        6074
C1200        4837
C3000        1918
C2100        1883
MidOther     1458
C7000         777
LowOther       26
Name: CLASSIFICATION, dtype: int64

In [9]:
# Look at INCOME AMT value counts for binning
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [10]:
# Create function to bin specified column at specified cutoff constant
# Function in this cell is used to bin the income amount column at a cutoff of 500
def binner(column, cutoff):
    application_types_to_replace = []
    for item in application_df[column].value_counts().items():
        if item[1] < cutoff:
            application_types_to_replace.append(item[0])

    # Replace in dataframe
    for app in application_types_to_replace:
        application_df[column] = application_df[column].replace(app,"Other")

binner("INCOME_AMT", 500)

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(data=application_df, columns=list(set(['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION',
                                                             'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS',
                                                             'INCOME_AMT'])-set(drop_columns)))

In [12]:
converted_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_LowOther,...,ORGANIZATION_Trust,APPLICATION_TYPE_LowOther,APPLICATION_TYPE_MidOther,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,1,142590,1,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
converted_data.shape

(34299, 44)

In [14]:
# Split our preprocessed data into our features and target arrays
# y is target / x is features
y = converted_data['IS_SUCCESSFUL']
X = converted_data.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
X_test.shape

(8575, 43)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model
nn = Sequential()

# First hidden layer
nn.add(Dense(units=45, activation = 'relu', input_dim=X_test.shape[1]))

# Second hidden layer
nn.add(Dense(units=5, activation='sigmoid'))

# Third hidden layer
nn.add(Dense(units=15, activation='sigmoid'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 45)                1980      
                                                                 
 dense_1 (Dense)             (None, 5)                 230       
                                                                 
 dense_2 (Dense)             (None, 15)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 2316 (9.05 KB)
Trainable params: 2316 (9.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Create a checkpoint to save the best output
checkpoint = ModelCheckpoint("best_model_checkpoint_optimized.h5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=30, callbacks=[checkpoint])

Epoch 1/30
  1/804 [..............................] - ETA: 2:15 - loss: 1.0363 - accuracy: 0.5312
Epoch 1: accuracy improved from -inf to 0.46250, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250
 67/804 [=>............................] - ETA: 0s - loss: 1.0240 - accuracy: 0.4487  
Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46250

Epoch 1: accuracy did not improve from 0.46

/Users/owner/anaconda3/envs/dev/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: accuracy improved from 0.49541 to 0.49950, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.49950 to 0.50478, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.50478 to 0.50913, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.50913 to 0.51309, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.51309 to 0.51736, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.51736 to 0.52216, saving model to best_model_checkpoint_optimized.h5
279/804 [=========>....................] - ETA: 0s - loss: 0.7858 - accuracy: 0.5250
Epoch 1: accuracy improved from 0.52216 to 0.52589, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.52589 to 0.52895, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.52895 to 0.53222, saving model to best_model_chec


Epoch 1: accuracy improved from 0.62929 to 0.63038, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63038 to 0.63140, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63140 to 0.63201, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63201 to 0.63285, saving model to best_model_checkpoint_optimized.h5
625/804 [======================>.......] - ETA: 0s - loss: 0.6854 - accuracy: 0.6328
Epoch 1: accuracy improved from 0.63285 to 0.63368, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63368 to 0.63435, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63435 to 0.63516, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63516 to 0.63624, saving model to best_model_checkpoint_optimized.h5

Epoch 1: accuracy improved from 0.63624 to 0.63692, saving model to best_model_chec


Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129
311/804 [==========>...................] - ETA: 0s - loss: 0.5715 - accuracy: 0.7285
Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epoch 2: accuracy did not improve from 0.74129

Epo

...


Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375
753/804 [===========================>..] - ETA: 0s - loss: 0.5424 - accuracy: 0.7359
Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375

Epoch 26: accuracy did not improve from 0.84375
804/804 [==============================] - 0s 537us/step - loss: 0.5431 - accuracy: 0.7354
Epoch 27/30

Epoch 27: accuracy improved from 0.84375 to 0.87500, saving model to best_model_checkpoint_optimized.h5
  1/804 [..............................] - ETA: 5s - loss: 0.3944 - accuracy: 0.8750
Epoch 27: accuracy did not improve 


Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500
752/804 [===========================>..] - ETA: 0s - loss: 0.5427 - accuracy: 0.7360
Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improve from 0.87500

Epoch 27: accuracy did not improv


Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500
657/804 [=======================>......] - ETA: 0s - loss: 0.5422 - accuracy: 0.7360
Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improve from 0.87500

Epoch 28: accuracy did not improv


Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500
655/804 [=======================>......] - ETA: 0s - loss: 0.5434 - accuracy: 0.7361
Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improve from 0.87500

Epoch 29: accuracy did not improv


Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500
559/804 [===================>..........] - ETA: 0s - loss: 0.5415 - accuracy: 0.7328
Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improve from 0.87500

Epoch 30: accuracy did not improv

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5495 - accuracy: 0.7317 - 134ms/epoch - 500us/step
Loss: 0.5494785904884338, Accuracy: 0.7316617965698242


In [22]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_optimized.h5')